## 📸 Vision

This module provides the necessary global perception for the system, transforming raw camera images into accurate world coordinates for path planning and state estimation correction ($\text{vision.py}$).

### Theory: Perspective Transformation and ArUco Detection

* **Perspective Transformation:** A homography matrix ($\text{perspective\_matrix}$) is computed using $\text{cv2.getPerspectiveTransform}$. This matrix maps four known source points (the calibration markers defining the map corners) in the camera image to four desired destination points in a rectified, birds-eye view image space. This corrects the distortion from the overhead camera's perspective.
* **Robot Pose Estimation:** The robot's $\text{Pose}(x, y, \theta)$ is determined by detecting a single **ArUco marker** attached to the robot. The center of the marker gives the $(x, y)$ position, and the orientation of the marker's edges gives the angle $\theta$.

### Implementation Details ($\text{vision.py}$)

* **Calibration:** The `_detect_calibration_aruco` method finds the corner markers and sorts them to establish the correct spatial order [Top-Left, Top-Right, Bottom-Left, Bottom-Right] required by `cv2.getPerspectiveTransform`.
* **Coordinate System:** The output pose ($\text{x\_cm, y\_cm}$) is carefully converted from image pixels to centimeters. The image Y-axis (downwards) is flipped and scaled to match the world Y-axis (upwards) before returning the $\text{Pose}$.
* **Camera Warmup:** The `warmup_camera` function waits until the mean brightness of frames is sufficiently high ($\text{mean\_brightness} > 10$), mitigating auto-exposure delays and ensuring the first processed frames are stable and correctly exposed.

#### Code: Vision System Module

```python
# CODE FROM vision.py (Demonstrates core structure)
import cv2
import numpy as np
# ... (rest of vision.py content)

class VisionSystem:
    # ... (init, release, warmup_camera, _detect_calibration_aruco methods)

    def warp_image(self, img: np.ndarray) -> np.ndarray:
        """Applies the perspective transformation to the image."""
        return cv2.warpPerspective(img, self.perspective_matrix, (self.map_width_pxl, self.map_height_pxl))

    def get_robot_pose(self) -> Optional[Pose]:
        """
        Captures an image, warps it, and detects the robot's pose using ArUco markers.
        (Full implementation includes pixel-to-cm conversion and orientation calculation.)
        """
        # ... (Robot pose detection logic)
        pass

## 🔬 Filtering (EKF) [To modify bc I didnt read the code enough yet]

This module provides a smooth and accurate estimate of the robot's pose by fusing the noisy odometry model with the noisy vision measurements.

### Theory: Extended Kalman Filter (EKF)

The EKF is necessary because the robot's motion model (kinematics) and the vision measurements are both subject to noise. Furthermore, the robot's motion is governed by non-linear equations (due to the $\sin(\theta)$ and $\cos(\theta)$ terms), requiring the use of the Extended version of the Kalman Filter.

The EKF proceeds in two steps per cycle:

1.  **Prediction (Time Update):** Uses the robot's kinematic model ($\text{Thymio.state\_extrapolation\_f}$) to project the current pose ($\hat{x}_{k-1}$) and covariance ($\hat{P}_{k-1}$) forward in time, accounting for the process noise ($Q$).
2.  **Update (Measurement Update):** Corrects the predicted pose ($\bar{x}_k$) using the measurement ($\text{z}_k$) from the vision system, weighted by the Kalman Gain ($K$).

### Kinematic Model and Noise ($\text{thymio\_math\_model.py}$)

The $\text{Thymio}$ class defines the differential-drive kinematic equations and computes the Process Noise Covariance matrix ($Q$).

* **Process Noise ($Q$):** $Q$ is calculated dynamically using the Jacobian ($G$) of the state transition function with respect to the input (wheel speeds, $v_l, v_r$).
    $$Q = G \Sigma_u G^T + Q_{\text{model}}$$
    The input covariance $\Sigma_u$ accounts for noise proportional to the wheel speed: $\text{var}(v) \propto |u|$, which is essential for realistic noise modeling in small robots.
* **Parameters:** The constants $L$ ($9.5 \text{ cm}$ axle length), $R$ ($2.2 \text{ cm}$ wheel radius), and the noise scaling factors ($\text{kvl} = 0.47, \text{kvr} = 0.51$) were obtained through physical measurements and empirical data analysis of the Thymio.

#### Code: EKF Initialization

```python
# CODE FROM state_estimation.py and thymio_math_model.py
import numpy as np
from utils import Pose
# ... (rest of state_estimation.py and thymio_math_model.py content)

## ⚙️ Motion Control Module Implementation

The robot's motion control is executed by the ThymioController class, which operates as a **Finite State Machine (FSM)**. This structure allows for seamless and stable arbitration between global, goal-oriented driving and local, reactive obstacle avoidance.

---

### 1. FSM and Control Strategy: Blending and Stability

#### Theory: FSM with Time-Based Control Blending

The core design principle is the use of a time-based blending mechanism when transitioning from the AVOIDING state back to NAVIGATING. This technique is a critical engineering choice to prevent control signal instability (overshooting or oscillation) upon re-entry to the complex proportional goal-seeking behavior.

The transition is governed by a fixed-duration cooldown phase (AVOIDANCE_DURATION = $\mathbf{4.0\text{ s}}$). During this time, the output motor commands are a linear interpolation between the stable, simple command (Drive Straight) and the complex navigation command (Move to Point).

The blending factor, $\text{decay}$, ensures a smooth ramp-down of the straight-drive influence:
$$\text{Command} = \text{decay} \cdot \text{Drive}_{\text{Straight}} + (1.0 - \text{decay}) \cdot \text{Command}_{\text{Navigate}}$$
where $\text{decay} \to 1.0$ at the start of the cooldown and $\text{decay} \to 0.0$ at the end.

#### Choices and Justification

* Fixed Cooldown Duration ($\mathbf{4.0\text{ s}}$): This fixed time, rather than a proportional distance or angular correction, was chosen for its robustness in the constrained Thymio environment. It guarantees sufficient time for the robot to clear the immediate disturbance and for the external pose estimation system to provide settled, stable data.
* Avoidance Threshold ($\mathbf{400}$): This low $\text{SENSOR\_THRESHOLD}$ was selected to trigger avoidance preemptively (at a greater distance). Early reaction is preferred to aggressive, late braking, especially given the high $\text{AVOID\_SPEED}$ ($\mathbf{125.0}$).

---

### 2. Goal-Oriented Driving: $\text{\_move\_to\_point()}$

This module forms the core of the NAVIGATING state, calculating the motor commands required to eliminate the angular error relative to the next global waypoint.

#### Theory: P-Controller with Regulated Linear Speed

The controller uses a Proportional (P) Controller for angular control, coupled with a regulated linear speed function based on the $\cos$ of the heading error.

* Angular Control ($\omega$): $\omega$ is proportional to the $\text{heading\_error}$ ($\omega = \text{KP\_ROT} \times \text{heading\_error}$).
* Linear Speed ($V$): $V$ is attenuated by $\max(0.0, \cos(\text{heading\_error}))$. This design ensures the robot slows down when it needs to make large turning corrections (i.e., when $\text{heading\_error}$ approaches $\pm 90^\circ$), resulting in a stable, efficient turn-in-place maneuver instead of a large arcing path.

#### Parameter Tuning

| Parameter | Value | Rationale for Choice |
| :--- | :--- | :--- |
| $\text{KP\_ROT}$ | $\mathbf{120.0}$ | A high proportional gain, empirically tuned for quick, decisive steering with minimal oscillation around the target heading. |
| $\text{MAX\_SPEED}$ | $\mathbf{125.0}$ | Set high to ensure high path efficiency when driving straight, matching the $\text{AVOID\_SPEED}$. |

---

### 3. Reactive Avoidance: $\text{\_avoid\_obstacles()}$

This function generates the motor commands during the AVOIDING state, taking over path following to prevent collision with a local obstacle.

#### Theory: Differential Steering with Dynamic Weighting

The avoidance mechanism is a differential steering technique where the $\text{turn}_{\text{weight}}$ is derived from the weighted sensor inputs.

A key design innovation is the use of dynamic sensor weighting based on the symmetry of the frontal threat:

$$\text{turn}_{\text{weight}} = (S_0 \cdot W_{\text{outer}} + S_1 \cdot W_{\text{inner}}) - (S_3 \cdot W_{\text{inner}} + S_4 \cdot W_{\text{outer}})$$

If the inner sensors ($\text{sensor}[1]$ and $\text{sensor}[3]$) are roughly equal ($\text{symmetric threat}$), the controller forces a dramatic increase in the weight of the outer sensors ($W_{\text{outer}} \to 2.3$, $W_{\text{inner}} \to 0$), compelling the robot to commit quickly to the side with the slightest opening. Otherwise, standard weights ($\mathbf{1.7}$ inner, $\mathbf{1.0}$ outer) are used for finer correctional steering.

#### Parameter Tuning

| Parameter | Value | Rationale for Choice |
| :--- | :--- | :--- |
| $\text{AVOID\_SPEED}$ | $\mathbf{125.0}$ | The avoidance speed is intentionally set equal to $\text{MAX\_SPEED}$. This decision prioritizes clearing the obstacle area quickly, relying on the aggressive $\text{angular\_speed}$ calculation to manage the turn rather than slowing down. |
| $\text{Rotation Gain}$ | $\mathbf{0.025}$ | A low gain applied to the $\text{turn}_{\text{weight}}$. This is necessary because the $\text{turn}_{\text{weight}}$ can be a very large number (sum of sensor readings). The low gain scales the powerful sensor input into a stable, non-spinning rotational impulse. |

## Code

```python
# CODE FROM control.py (Modified for independent execution)
import math
from utils import Point, Pose, angle_between, normalize_angle, RobotState
import time

class ThymioController:
    def __init__(self):
        self.KP_ROT = 120.0  # P-gain for rotation
        self.MAX_SPEED = 125.0  # Thymio units
        self.AVOID_SPEED = 125.0
        self.SENSOR_THRESHOLD = 400  # Threshold to trigger avoidance
        self.AVOIDANCE_DURATION = 4.0  # Number of seconds to stay in avoidance state
        self.FRONT_SENSOR_MIN_DELTA = 30
        self.last_avoidance_time = 0.0

        # Internal State
        self.state: RobotState = RobotState.NAVIGATING

    def update(self, current_pose: Pose, target_pos: Point, sensor_data: list[int]) -> tuple[float, float]:
        """
        FSM and transition logic with time-based blending.
        """
        current_time = time.time()

        max_front_sensor = max(sensor_data[0:5])
        is_obstacle_present = max_front_sensor > self.SENSOR_THRESHOLD

        nav_l, nav_r = self._move_to_point(current_pose, target_pos)

        if is_obstacle_present:
            # State 1: IMMEDIATE AVOIDANCE
            self.state = RobotState.AVOIDING
            self.last_avoidance_time = current_time
            return self._avoid_obstacles(sensor_data)

        elif self.state == RobotState.AVOIDING:
            # State 2: TIME-BASED COOLDOWN AND BLENDING
            elapsed = current_time - self.last_avoidance_time
            if elapsed < self.AVOIDANCE_DURATION:
                
                # Calculate decay factor (1.0 at start of cooldown, 0.0 at end)
                decay = max(0.0, 1.0 - (elapsed / self.AVOIDANCE_DURATION))

                # Drive straight component (target of the blend)
                straight_l = self.MAX_SPEED
                straight_r = self.MAX_SPEED

                # Blend controls: decay (straight) + (1.0 - decay) (navigation)
                l_cmd = decay * straight_l + (1.0 - decay) * nav_l
                r_cmd = decay * straight_r + (1.0 - decay) * nav_r

                return int(l_cmd), int(r_cmd)
            else:
                # Cooldown finished
                self.state = RobotState.NAVIGATING

        # Default: Navigation
        return nav_l, nav_r

    def _move_to_point(self, current_pose: Pose, target_pos: Point) -> tuple[float, float]:
        """
        P-Controlled navigation to a target point.
        """
        angle_to_target = angle_between(current_pose, target_pos)
        heading_error = normalize_angle(angle_to_target - current_pose.theta)
        angular_speed = self.KP_ROT * heading_error
        linear_speed = self.MAX_SPEED * max(0.0, math.cos(heading_error))

        l_speed = linear_speed - angular_speed
        r_speed = linear_speed + angular_speed

        return int(l_speed), int(r_speed)

    def _avoid_obstacles(self, sensor_data: list[int]) -> tuple[float, float]:
        """
        Differential steering for evasive maneuvers with dynamic sensor weighting.
        """
        
        inner_sensor_weight = 1.7
        outer_sensor_weight = 1.0
        
        # Dynamic Weighting Logic: if frontal threat is symmetric, prioritize outer sensors
        if abs(sensor_data[1] - sensor_data[3]) < self.FRONT_SENSOR_MIN_DELTA:
            inner_sensor_weight = 0.0
            outer_sensor_weight = 2.3

        turn_weight = (
            sensor_data[0] * outer_sensor_weight
            + sensor_data[1] * inner_sensor_weight
            - sensor_data[3] * inner_sensor_weight
            - sensor_data[4] * outer_sensor_weight
        )

        angular_speed = turn_weight * 0.025

        l_speed = self.AVOID_SPEED + angular_speed
        r_speed = self.AVOID_SPEED - angular_speed

        return int(l_speed), int(r_speed)

## 🌎 Global Navigation

This module is responsible for defining the static environment and calculating the shortest, obstacle-free path for the robot's navigation (using $\text{pathfinding.py}$ and map data from $\text{vision.py}$).

### Theory: Visibility Graph and A\* Algorithm

The global path planning strategy uses a **Visibility Graph (VG)** combined with the **A\* search algorithm**.

* **Visibility Graph Construction:** The VG's nodes consist of all detected obstacle vertices (after C-Space buffering) plus the robot's starting position and the final goal position. An edge exists between any two nodes if the straight-line segment connecting them is visible.
* **A\* Search:** A\* is an **informed search algorithm** used to traverse the VG. It finds the path minimizing the total estimated cost, $f(n)$, where:
    $$
    f(n) = g(n) + h(n)
    $$
    $g(n)$ is the actual distance traveled, and $h(n)$ is the Euclidean distance heuristic to the goal. A\* guarantees finding the globally shortest path on the graph.

### Implementation Details ($\text{vision.py}$ and $\text{pathfinding.py}$)

* **Obstacle Configuration Space (C-Space):** A critical step in $\text{vision.py}$ is buffering all static obstacles by a margin ($\text{ROBOT\_RADIUS\_CM} + \text{padding}$) to ensure the resulting path avoids collision with the robot's physical body.
* **Graph Augmentation ($\text{add\_robot\_to\_graph}$):** Since the robot's start position changes, the $\text{base\_graph}$ (obstacles + goal) is dynamically augmented by adding the robot's current EKF-filtered $\text{Pose}$ as the temporary start node, ensuring the path is always computed from the robot's latest location.

#### Code: Pathfinding Module

```python
# CODE FROM pathfinding.py (Used to execute the module independently)
import heapq
import networkx as nx
from utils import Point, HeuristicFunction, euclidean_distance

def reconstruct_path(came_from: dict[int, int], current: int) -> list[int]:
    """
    Backtracks from a goal node (ID) to start node (ID) to generate the final path list.
    """
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path.append(current)
    return total_path[::-1]

def find_path(
    graph: nx.Graph,
    start_node: int,
    goal_node: int,
    heuristic: HeuristicFunction = euclidean_distance,
) -> list[int]:
    # ... (Full implementation of A* algorithm using heapq for priority queue management)
    pass

## 🧭 Local Navigation: Reactive Obstacle Avoidance

This module provides an **immediate, priority control signal** whenever an obstacle is detected, overriding the global goal-seeking behavior of the navigation system. The local navigation mechanism is structured into an **Active Evasion Phase** and a **Blended Cooldown Phase**.

---

### Theory: Differential Steering and Control Blending

The system transitions between two distinct control modes:

1.  **Active Evasion:** This mode is triggered immediately when proximity sensor readings exceed the threshold ($\text{SENSOR\_THRESHOLD} = \mathbf{400}$). The robot calculates a rotational impulse based on the threat's proximity, and the goal is to induce a rapid pivot away from the side experiencing the highest sensor threat.

2.  **Blended Cooldown:** Upon clearing the obstacle, the robot enters a **time-based decay phase** of $\mathbf{4.0\text{ seconds}}$ ($\text{AVOIDANCE\_DURATION}$). This mechanism serves two purposes: preventing the robot from steering back into the obstacle's path and ensuring a smooth, non-oscillatory re-entry to the navigation path by avoiding a sudden oversteer back towards the target.

#### Evasion Drive Calculation

The core of the avoidance mechanism is **differential steering**, where the $\text{turn}_{\text{weight}}$ is derived from the threat imbalance and applied symmetrically to the base speed ($\text{AVOID}_{\text{SPEED}}$):

$$
L_{\text{speed}} = \text{AVOID}_{\text{SPEED}} + \text{turn}_{\text{weight}} \times \text{gain}
$$
$$
R_{\text{speed}} = \text{AVOID}_{\text{SPEED}} - \text{turn}_{\text{weight}} \times \text{gain}
$$

A **positive** $\text{turn}_{\text{weight}}$ (indicating an obstacle on the left) results in $L_{\text{speed}}$ increasing and $R_{\text{speed}}$ decreasing, forcing a turn **right**. The opposite action occurs for a negative $\text{turn}_{\text{weight}}$.

### Implementation Details

* **Turning Weight Calculation (Dynamic):** The $\text{turn}_{\text{weight}}$ is calculated using a **dynamic sensor weighting** strategy to adapt to different obstacle geometries. Base weights ($\mathbf{1.7}$ inner, $\mathbf{1.0}$ outer) are used for general threats. However, if the threat is frontal/symmetric ($\text{sensor}[1] \approx \text{sensor}[3]$), inner sensors are ignored ($\mathbf{0.0}$) and outer sensors are heavily prioritized ($\mathbf{2.3}$) to force a sharp directional evasion.

    The weighted difference is computed as:
    $$
    \text{turn}_{\text{weight}} = (S_0 \cdot W_{\text{outer}} + S_1 \cdot W_{\text{inner}}) - (S_3 \cdot W_{\text{inner}} + S_4 \cdot W_{\text{outer}})
    $$

* **Blend Logic:** During the $\mathbf{4.0\text{ s}}$ cooldown, the blend factor ($\text{decay}$) interpolates the motor commands:

    $$\text{Command} = \text{decay} \cdot \text{Straight} + (1.0 - \text{decay}) \cdot \text{Navigate}$$

    This ensures the robot gradually shifts from prioritizing simple forward stability (when $\text{decay} \approx 1.0$) back to complex goal-seeking control (when $\text{decay} \approx 0.0$).

# Code:

```python
# CODE FROM control.py (Demonstrates avoidance and blending logic)
# This code is executed in the update loop by the FSM.
import time
# ... (imports and class definition)

class ThymioController:
    # ... (relevant __init__ variables: AVOIDANCE_DURATION, SENSOR_THRESHOLD, etc.)

    def update(self, current_pose: Pose, target_pos: Point, sensor_data: list[int]) -> tuple[float, float]:
        # ... (transition logic, including time-based elif self.state == RobotState.AVOIDING block)
        
        # When blending:
        # decay = max(0.0, 1.0 - (elapsed / self.AVOIDANCE_DURATION))
        # l_cmd = decay * straight_l + (1.0 - decay) * nav_l
        # r_cmd = decay * straight_r + (1.0 - decay) * nav_r
        # ...

    def _avoid_obstacles(self, sensor_data: list[int]) -> tuple[float, float]:
        """
        Differential steering for evasive maneuvers with dynamic sensor weighting.
        """
        # ... (dynamic weighting calculation using inner_sensor_weight, outer_sensor_weight)
        
        # turn_weight calculation:
        # turn_weight = (S0 * W_outer + S1 * W_inner - S3 * W_inner - S4 * W_outer)

        # angular_speed = turn_weight * 0.025

        # l_speed = self.AVOID_SPEED + angular_speed
        # r_speed = self.AVOID_SPEED - angular_speed

        # return int(l_speed), int(r_speed)
        pass # Placeholder for full code execution